Written by [Gameli Ladzekpo](mailto:gameli.Ladzekpo@gmail.com) (Twitter/IG: @gamladz)

For [AI Core](theaicore.com)

In [2]:
First Import the files

In [48]:
# Start with imports 

import json 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as expected_conditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
from time import sleep, time
import random
import re
import subprocess, os
import urllib.request

In [31]:
# open_chrome function
def open_chrome(port=9220, on_mac=True):
    my_env = os.environ.copy()
    if on_mac:
        subprocess.Popen(['open', '-a', "Google Chrome", '--args', f'--remote-debugging-port={port}', 'http://www.example.com'], env=my_env)
    else:
        subprocess.Popen(f'google-chrome --remote-debugging-port={port} --user-data-dir=bots'.split(), env=my_env)

class Bot():
    def __init__(self, port_no = 9220, headless = False, verbose = False):
        print('initialising bot')

        open_chrome()

        options = Options()
        options.add_argument("--no-sandbox")	# without this, the chrome webdriver can't start (SECURITY RISK)
        options.add_experimental_option(f"debuggerAddress", f"127.0.0.1:{port_no}")	# attach to the same port that you're running chrome on
        if headless:
            options.add_argument("--headless") # headless option allows scraper to run in the background
        #options.add_argument("--window-size=1920x1080")
        self.driver = webdriver.Chrome('chrome_driver/chromedriver')			# create webdriver
        self.verbose = verbose

    def click_btn(self, text):
        if self.verbose: print(f'clicking {text} btn')
        element_types = ['button', 'div', 'input', 'a', 'label']
        
        for element_type in element_types:
            btns = self.driver.fund_elements_by_xpath(f'//{element_type}')
            # for btn in btns:
            #   print(btn.text)

            # SEARCH BY TEXT
            try:
                btn = [b for b in btns if b.text.lower() == text.lower()][0]
                btn.click()
                return
            except IndexError:
                pass

            # SEARCH BY VALUE ATTRIBUTE IF NOT YET FOUND
            try:
                btn = self.driver.find_elements_by_xpath(f'//{element_type}[@value="{text}"]')[0]
                btn.click()
                return
            except:
                continue

        raise ValueError(f'button containing "{text}" not found')

    def _search(self, query, _type='search', placeholder=None):
        sleep(1)
        s = self.driver.find_elements_by_xpath(f'//input[@type="{_type}"]')
        print(s)
        if placeholder:
            s = [i for i in s if i.get_attribute('placeholder').lower() == placeholder.lower()][0]
        else:
            s = s[0]
        s.send_keys(query) 

    def toggle_verbose(self):
        self.verbose = not self.verbose

In [110]:
from selenium import webdriver 

if __name__ == '__main__':
    # EXAMPLE USAGE
    bot = Bot()

    searches = ['plant pots','plates', 'bin']
    for search in searches:
        bot.driver.get(f'https://www.ikea.com/gb/en/search/products/?q={search}')

        # Go into the main grid and find the link for each result
        results = bot.driver.find_elements_by_xpath('//*[@id="search-results"]/div/a')  
        print (f'found {len(results)}) results for search "{search}" ')

        results = [r.get_attribute('href') for r in results] 

        for result in results:
            bot.driver.get(result)
            result = result.split('/')[-2]

            # Product Dimensions

            # First get the image - product dimensions image
            prod_dims_images = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-product-dimensions-content__images"]//img')
            prod_dims_images = [i.get_attribute('src') for i in prod_dims_images]

            for image in prod_dims_images:
                print (prod_dims_images)

                for idx, img_url in enumerate(prod_dims_images):
                    filename = f'dims-{result}-{idx}'
                    file_ext = img_url.split('.')[-1] 
                    file_ext = file_ext[ 0 : 3 ]
                    urllib.request.urlretrieve(img_url, f'data/{result}/{filename}.{file_ext}')

        



            # Cycle through information on each page starting with images   
            images = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-media-grid__media-container"]//img') 
            images = [i.get_attribute('src') for i in images]

            # Product Details
            product_details = bot.driver.find_elements_by_xpath('//*[@class="range-revamp-product-details__paragraph"]')
            print (len(product_details))

            product_details = [paragraph.get_attribute('innerHTML') for paragraph in product_details]
            for paragraph in product_details:
                print (paragraph)  
            

            

            os.makedirs(f'data/{result}', exist_ok=True)


            # Get reviews
            #
            
            for image in images:
                # Loop through each image and save to disk

                for idx, img_url in enumerate(images):

                    # Create filename for each with ID and get the file extension
                    filename = f'{result}-{idx}'
                    file_ext = img_url.split('.')[-1] 
                    file_ext = file_ext[ 0 : 3 ]
                    # Write to file
                    urllib.request.urlretrieve(img_url, f'data/{result}/{filename}.{file_ext}')


 
          
            



        


initialising bot
found 31) results for search "plant pots" 
['https://www.ikea.com/gb/en/images/products/muskot-plant-pot-white__0595931_PE717288_S5.JPG?f=xs']
3
Decorate your home with plants combined with a plant pot to suit your style.
May be combined with the other plant pots in the MUSKOT series.
To protect the surface from any risk of moisture damage, always use a plant mover or put felt pads under the plant pot or saucer.
['https://www.ikea.com/gb/en/images/products/boysenbaer-plant-pot-in-outdoor-light-grey__0674252_PE717650_S5.JPG?f=xs']
5
This pot is made of concrete with natural air bubbles and variations in the finish that make each pot unique.  
The pot has a soft pad underneath that protects the underlaying surface from scratches and moisture.
The plant pot is frost-resistant and can be kept outdoors at temperatures below zero if emptied of soil or covered.
Suitable for both indoor and outdoor use.
Make a drain hole in the pot with a drill, if needed.
['https://www.ikea.c

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=88.0.4324.96)
